from prody import *
from pylab import *

In [ ]:
pdb, header = parsePDB('6acg', header=True)

In [34]:
contacts = pdb.select('chain A B C and within 4 of chain D')

In [35]:
contacts

<Selection: 'chain A B C and...in 4 of chain D' from 6acg (46 atoms)>